In [ ]:
import cv2
import json
import matplotlib.pyplot as plt
from collections import deque
from datetime import datetime
import os
from ultralytics import YOLO
import matplotlib

# Ensure Matplotlib works in all environments
matplotlib.use('TkAgg')

# -------------------- CONFIGURATION --------------------
VIDEO_PATH = 'model2/v1.mp4'         # Change this to your video file path
YOLO_MODEL = 'yolov8n.pt'            # Use a known working YOLOv8 model

FRAME_SKIP = 3                       # Detect every 3rd frame
CONSEC_FRAMES = 2                   # Reduce for faster testing
PEOPLE_THRESHOLD = 1                # Reduce to trigger alert more easily

EXPORT_VIDEO = True                  # Save output video
SHOW_GUI = True                      # Display live simulation window
RESIZE_DIMS = (640, 360)             # Resize frame for speed

OUTPUT_VIDEO = 'output_with_alerts.mp4'
ALERT_JSON = 'alert_log.json'
ALERT_TXT = 'alert_log.txt'

# -------------------- MODEL LOADING --------------------
def load_yolo_model(model_path):
    try:
        model = YOLO(model_path)
        print(f"✅ YOLO model loaded successfully with classes: {model.names}")
        return model
    except Exception as e:
        print(f"❌ Error loading YOLO model: {e}")
        raise

model = load_yolo_model(YOLO_MODEL)

# -------------------- VIDEO SETUP --------------------
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise FileNotFoundError(f"❌ Cannot open video at {VIDEO_PATH}")

fps = cap.get(cv2.CAP_PROP_FPS) or 25
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"📹 Video loaded: {VIDEO_PATH} | FPS: {fps} | Total Frames: {total_frames}")

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
frame_w, frame_h = RESIZE_DIMS

if EXPORT_VIDEO:
    out = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, RESIZE_DIMS)

# -------------------- VARIABLES --------------------
frame_count = 0
alert_log = []
alert_frames = []
recent_counts = deque(maxlen=CONSEC_FRAMES)

# -------------------- MAIN LOOP --------------------
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("✅ Finished processing all frames.")
        break

    frame_count += 1
    frame = cv2.resize(frame, RESIZE_DIMS)
    display_frame = frame.copy()

    if frame_count % FRAME_SKIP == 0:
        people_count = 0
        results = model(frame, verbose=False)

        for result in results:
            for box in result.boxes:
                cls = int(box.cls[0])
                class_name = model.names.get(cls, 'unknown')
                if class_name == 'person':
                    people_count += 1
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    cv2.rectangle(display_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(display_frame, 'Person', (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        recent_counts.append(people_count)
        print(f"📊 Frame {frame_count}: Detected {people_count} person(s)")

        # Check for alert condition
        if len(recent_counts) == CONSEC_FRAMES and all(p >= PEOPLE_THRESHOLD for p in recent_counts):
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            alert_log.append({
                'frame': frame_count,
                'timestamp': timestamp,
                'alert': 'Crowd Detected'
            })
            alert_frames.append(frame_count)
            print(f"🚨 ALERT triggered at frame {frame_count} | {timestamp}")
            cv2.putText(display_frame, 'ALERT: Crowd Detected!', (40, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)
        else:
            cv2.putText(display_frame, f'People Count: {people_count}', (40, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)

    # Show or save frame
    if EXPORT_VIDEO:
        out.write(display_frame)
    if SHOW_GUI:
        cv2.imshow("Real-Time Crowd Detection", display_frame)
        if cv2.waitKey(int(1000 / fps)) & 0xFF == ord('q'):
            print("❌ User terminated the program.")
            break

# -------------------- CLEANUP --------------------
cap.release()
if EXPORT_VIDEO:
    out.release()
if SHOW_GUI:
    cv2.destroyAllWindows()

# -------------------- LOGGING --------------------
# Save JSON
with open(ALERT_JSON, 'w') as f_json:
    json.dump(alert_log, f_json, indent=2)
print(f"✅ Alert log saved to {ALERT_JSON}")

# Save TXT
with open(ALERT_TXT, 'w') as f_txt:
    for entry in alert_log:
        f_txt.write(f"{entry['timestamp']} - Frame {entry['frame']} - {entry['alert']}\n")
print(f"✅ Alert log saved to {ALERT_TXT}")

# -------------------- PLOT TIMELINE --------------------
if alert_frames:
    plt.figure(figsize=(10, 2))
    plt.eventplot(alert_frames, orientation='horizontal', colors='red')
    plt.xlabel('Frame Number')
    plt.title('Crowd Alert Timeline')
    plt.tight_layout()
    plt.show()
else:
    print("ℹ️ No alerts triggered. No timeline to show.")


  0%|          | 0.00/6.25M [00:00<?, ?B/s]

✅ YOLO model loaded successfully with classes: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote'